In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
%config InlineBackend.figure_format = 'retina'

In [ ]:
def sheet_cmap(reverse=False):
    # a nice colormap
    one = "#f4f0e8".lstrip('#')
    two = "#383b3e".lstrip('#')
    one = tuple(int(one[i:i+2], 16)/255 for i in (0, 2, 4))
    two = tuple(int(two[i:i+2], 16)/255 for i in (0, 2, 4))
    if reverse:
        one, two = two, one
    colors = [one, two]
    cmap = mcolors.LinearSegmentedColormap.from_list('sheet_cmap', colors, N=256)
    return cmap

In [ ]:
# Configuration params
size = (2000,2000)
n_particles = 800000
rng = np.random.default_rng()

# field: 1 for moving particles, 0 otherwise
field = np.zeros(size[0]*size[1], dtype=np.int8).reshape(size)
particles_pos = np.unravel_index(np.random.choice(np.arange(size[0]*size[1]), n_particles, replace=False), size)
field[particles_pos] = 1

# sticked_field: 1 for sticked particles, 0 otherwise
sticked_field = np.zeros(size[0]*size[1], dtype=np.int8).reshape(size)
sticked_field[size[0]//2, size[1]//2] = 1

#array view for margolous blocking of field
field_b_even = field.reshape((-1, 2, field.shape[1]//2, 2))  
field_blocks_even = field_b_even.transpose((0,2,1,3))

field_b_odd = field[1:-1,1:-1].reshape((-1, 2, field.shape[1]//2-1, 2))  
field_blocks_odd = field_b_odd.transpose((0,2,1,3))

In [ ]:
#DLA simulation
n_iter = 10000
for i in range(n_iter):
    #randomly move particles
    if i%2==0:
        rng.shuffle(field_blocks_even, axis=3)
        rng.shuffle(field_blocks_even, axis=2)
    else:
        rng.shuffle(field_blocks_odd, axis=3)
        rng.shuffle(field_blocks_odd, axis=2)
    
    #check for sticking
    stick_1_0 = (field[2:,1:-1]+sticked_field[1:-1,1:-1]==2) & (sticked_field[2:,1:-1]==0)
    field[2:,1:-1][stick_1_0]=0
    sticked_field[2:,1:-1][stick_1_0]=1
    
    stick_m1_0 = (field[0:-2,1:-1]+sticked_field[1:-1,1:-1]==2) & (sticked_field[0:-2,1:-1]==0)
    field[0:-2,1:-1][stick_m1_0]=0
    sticked_field[0:-2,1:-1][stick_m1_0]=1
    
    stick_0_1 = (field[1:-1, 2:]+sticked_field[1:-1,1:-1]==2) & (sticked_field[1:-1, 2:]==0)
    field[1:-1, 2:][stick_0_1]=0
    sticked_field[1:-1, 2:][stick_0_1]=1
    
    stick_0_m1 = (field[1:-1, 0:-2]+sticked_field[1:-1,1:-1]==2) & (sticked_field[1:-1, 0:-2]==0)
    field[1:-1, 0:-2][stick_0_m1]=0
    sticked_field[1:-1, 0:-2][stick_0_m1]=1


In [ ]:
print(f"Sticked particles: {np.count_nonzero(sticked_field)} - total particles: {np.count_nonzero(sticked_field) + np.count_nonzero(field)}")

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))
fig.subplots_adjust(left=0, bottom=0, right=1, top=1, wspace=None, hspace=None)
ax.spines[['top', 'right', 'bottom', 'left']].set_visible(False)
ax.set_aspect('equal', 'box')
ax.set_xlim([0,size[0]])
ax.set_ylim([0,size[1]])
#ax.set_axis_off()
ax.imshow(sticked_field*2+field, cmap=sheet_cmap())
plt.savefig("./dla_fast_2.png", dpi=600)
plt.show()